# Data Processing

This project uses MIMIC III v1.4 as the main dataset. We choose the variable **hospital_expire_flag** in *ADMISSIONS.csv* file as target variable, and we want to predict the death risk for patients who enter ICU for the first time. Therefore, our main purpose in data processing part is to find variables that related to the target variable for the next part of data analysis. To find these variables, we mainly use four files:
*LABEVENTS.csv, ICUSTAYS.csv, ADMISSIONS.csv, D_LABITEMS.csv, PATIENTS.csv*.

#### Load libraries

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

#### Load datasets

In [142]:
icustays = pd.read_csv('ICUSTAYS.csv')
labevents = pd.read_csv('LABEVENTS.csv')
admissions = pd.read_csv('ADMISSIONS.csv')
d_labitems = pd.read_csv('D_LABITEMS.csv')
patients = pd.read_csv('PATIENTS.csv')

### Icustays Processing

Firstly, we want to implement icustays processing, obtaining the time of each patient entering and leaving ICU for the first time, and the length of stay in ICU. 

In [5]:
icustays.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [11]:
# Change the "INTIME" and "OUTTIME" to datetime format, sort the icustays datafrmae by "INTIME".

icustays['INTIME'] = pd.to_datetime(icustays['INTIME'])
icustays['OUTTIME'] = pd.to_datetime(icustays['OUTTIME'])
icustays.sort_values('INTIME', inplace = True)

In [19]:
# Delete rows with missing values in column "OUTTIME"

icustays.drop(icustays[np.isnan(icustays['OUTTIME'])].index, axis = 0, inplace = True)

In [20]:
# Obtain the intime, outtime for each patient to enter the icu for the first time
# Due to the sorting of intime, the data in the first row of each patient is the data entered into ICU for the first time

time_dict = {}
for i in icustays.index:
    if icustays['SUBJECT_ID'][i] not in time_dict.keys():
        time_dict[icustays['SUBJECT_ID'][i]] = {'INTIME' : icustays['INTIME'][i], 
                                                'OUTTIME' : icustays['OUTTIME'][i]}

### Labevents processing

Secondly, we want to implement labevents processing. Each row of the datasets represents the value of a certain item tested by a certain patient. A patient may have multiple tests for one item. For a certain item, we want to obtain the value of the patient's first test during the first time entering ICU.

In [22]:
labevents.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal


In [21]:
# Only keep patients who have been in ICU

labevents_icu = labevents[[(sub_id in time_dict.keys()) for sub_id in labevents['SUBJECT_ID']]]

In [23]:
# Change the "CHARTTIME" to datetime format, sort the labevents_icu datafrmae by "INTIME".

labevents_icu['CHARTTIME'] = pd.to_datetime(labevents_icu['CHARTTIME'])
labevents_icu.sort_values('CHARTTIME', inplace = True)

<ipython-input-23-8f56ebfffe01>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labevents_icu['CHARTTIME'] = pd.to_datetime(labevents_icu['CHARTTIME'])
/opt/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


For every patient and every item, obtain the first testing value during his first entry into ICU.

In [45]:
%%time

itemid_dict = {}
require_index = {}
for i in labevents_icu.index:
    if labevents_icu['CHARTTIME'][i] > time_dict[labevents_icu['SUBJECT_ID'][i]]['INTIME'] and labevents_icu['CHARTTIME'][i] < time_dict[labevents_icu['SUBJECT_ID'][i]]['OUTTIME']:
        if labevents_icu['SUBJECT_ID'][i] not in itemid_dict.keys():
            itemid_dict[labevents_icu['SUBJECT_ID'][i]] = []
        if labevents_icu['ITEMID'][i] not in itemid_dict[labevents_icu['SUBJECT_ID'][i]]:
            itemid_dict[labevents_icu['SUBJECT_ID'][i]].append(labevents_icu['ITEMID'][i])
            require_index[i] = 0

CPU times: user 13min 9s, sys: 4.19 s, total: 13min 13s
Wall time: 13min 17s


In [46]:
# Select required rows

labevents_unique = labevents_icu[[(index in require_index.keys()) for index in labevents_icu.index]]

In [61]:
# Obtain the HADM_ID dict, in labevents_unqiue, for each patient, the HADM_ID should be unique

HADM_dict = {}
for i in labevents_unique.index:
    if labevents_unique['SUBJECT_ID'][i] not in HADM_dict.keys():
        HADM_dict[labevents_unique['SUBJECT_ID'][i]] = labevents_unique['HADM_ID'][i]

In [60]:
labevents_unique

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
25582306,25692457,82574,118464.0,50868,2100-06-07 23:34:00,13,13.00,mEq/L,NaN
25582304,25692455,82574,118464.0,50862,2100-06-07 23:34:00,3.2,3.20,g/dL,abnormal
25582305,25692456,82574,118464.0,50863,2100-06-07 23:34:00,75,75.00,IU/L,NaN
25912798,25692483,82574,118464.0,51301,2100-06-07 23:34:00,2.0,2.00,K/uL,abnormal
25582307,25692458,82574,118464.0,50878,2100-06-07 23:34:00,56,56.00,IU/L,abnormal
...,...,...,...,...,...,...,...,...,...
26709581,27638217,98185,116667.0,50813,2208-08-20 03:38:00,0.9,0.90,mmol/L,NaN
26709580,27638216,98185,116667.0,50808,2208-08-20 03:38:00,1.05,1.05,mmol/L,abnormal
26709579,27638215,98185,116667.0,50804,2208-08-20 03:38:00,26,26.00,mEq/L,NaN
26709578,27638214,98185,116667.0,50802,2208-08-20 03:38:00,0,0.00,mEq/L,NaN


Since the labevents are unique for each subject, we can use pd.pivot to convert the labevents_unique dataframe.

In [47]:
# Convert labevents_unique

convert = labevents_unique.pivot(index='SUBJECT_ID',columns='ITEMID',values='VALUE')
convert

ITEMID,50800,50801,50802,50803,50804,50805,50806,50808,50809,50810,...,51511,51512,51513,51514,51515,51516,51517,51518,51519,51523
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,
3,ART,403,-4,15,23,NaN,114,1.09,140,30,...,NaN,NaN,NaN,1,NaN,35,NaN,FEW,MANY,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NEG,NaN,0,NaN,NaN,NONE,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99985,ART,531,0,NaN,26,NaN,NaN,NaN,NaN,NaN,...,NaN,OCC,NaN,NEG,NaN,8,NaN,NaN,NONE,NaN
99991,NaN,506,0,NaN,23,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Since there are too many nan in convert, we want to keep the columns with missing values less than 20000, then delete the rows that has missing values in these columns.

In [49]:
drop_list = []
for column in convert.columns:
    if sum(pd.isna(convert[column])) > 20000:
        drop_list.append(column)
convert_dropna = convert.drop(drop_list, axis = 1).dropna(axis = 0, how = 'any')

In [44]:
len(drop_list)

671

In [50]:
convert_dropna

ITEMID,50802,50804,50818,50820,50821,50868,50882,50893,50902,50912,...,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,
3,-4,23,40,7.35,20,23,13,5.8,111,2.4,...,1.7,28.9,31.3,92,190,15.7,58.3,15.0,2.70,11.3
9,1,29,46,7.39,75,13,26,9.2,100,1.4,...,1.2,28.7,34.5,83,249,13.5,21.9,13.8,4.50,13.7
12,-21,10,36,7.02,276,28,11,8.3,111,1.3,...,1.8,30.1,33.7,90,87,16.8,60.9,14.2,4.40,8.4
13,0,26,43,7.38,369,14,23,8.9,106,0.6,...,1.2,30.2,34.6,87,216,13.7,44.1,13.4,4.08,16.6
17,-7,34,167,6.88,320,12,23,7.9,112,0.8,...,1.5,30.8,35.2,88,138,14.9,31.8,12.6,3.42,10.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99946,4,30,42,7.45,94,15,26,9.1,105,0.8,...,3.0,30.2,33.8,89,231,30.9,39.0,13.8,5.63,15.4
99982,8,33,40,7.51,100,13,30,8.8,98,1.5,...,1.3,33.0,33.8,98,113,14.7,31.4,15.2,3.18,3.8
99985,0,26,39,7.41,39,12,24,7.2,104,0.9,...,1.1,29.5,32.9,90,250,12.5,47.7,14.2,3.18,12.6


Some of the value cannot be transfered to float, therefore, we want to delete the rows with these string values.

In [51]:
def isfloat(i):
    try:
        float(i)
        return True
    except:
        return False
    
drop_index = []
for i in convert_dropna.index:
    for column in convert_dropna.columns:
        if not isfloat(convert_dropna[column][i]):
            drop_index.append(i)
            
convert_float = convert_dropna.drop(drop_index,axis = 0)

In [53]:
convert_float

ITEMID,50802,50804,50818,50820,50821,50868,50882,50893,50902,50912,...,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,
3,-4,23,40,7.35,20,23,13,5.8,111,2.4,...,1.7,28.9,31.3,92,190,15.7,58.3,15.0,2.70,11.3
9,1,29,46,7.39,75,13,26,9.2,100,1.4,...,1.2,28.7,34.5,83,249,13.5,21.9,13.8,4.50,13.7
12,-21,10,36,7.02,276,28,11,8.3,111,1.3,...,1.8,30.1,33.7,90,87,16.8,60.9,14.2,4.40,8.4
13,0,26,43,7.38,369,14,23,8.9,106,0.6,...,1.2,30.2,34.6,87,216,13.7,44.1,13.4,4.08,16.6
17,-7,34,167,6.88,320,12,23,7.9,112,0.8,...,1.5,30.8,35.2,88,138,14.9,31.8,12.6,3.42,10.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99946,4,30,42,7.45,94,15,26,9.1,105,0.8,...,3.0,30.2,33.8,89,231,30.9,39.0,13.8,5.63,15.4
99982,8,33,40,7.51,100,13,30,8.8,98,1.5,...,1.3,33.0,33.8,98,113,14.7,31.4,15.2,3.18,3.8
99985,0,26,39,7.41,39,12,24,7.2,104,0.9,...,1.1,29.5,32.9,90,250,12.5,47.7,14.2,3.18,12.6


### Admissions processing

We want to obtain the target variable and Admittime from admissions datasets.

In [139]:
admissions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [151]:
# Obtain the HADM_ID, flag, admittime for each patient

admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'])
admissions = admissions.sort_values('ADMITTIME')
HADM_dict = {}
admittime = {}
flag_dict = {}
for i in admissions.index:
    if admissions['SUBJECT_ID'][i] not in HADM_dict.keys():
        HADM_dict[admissions['SUBJECT_ID'][i]] = admissions['HADM_ID'][i]
        admittime[admissions['SUBJECT_ID'][i]] = admissions['ADMITTIME'][i]
        flag_dict[admissions['SUBJECT_ID'][i]] = admissions['HOSPITAL_EXPIRE_FLAG'][i]

### Patients processing

This dataset includes patients information, we want to get the date of birthday and gender for each patient.

In [144]:
patients['GENDER'] = patients['GENDER'].replace({'F':0, 'M':1})
patients['DOB'] = pd.to_datetime(patients['DOB'])

In [145]:
# Obtain the gender_dict and age_dict

gender_dict = {}
age_dict = {}
for i in patients.index:
    gender_dict[patients['SUBJECT_ID'][i]] = patients['GENDER'][i]
    age_dict[patients['SUBJECT_ID'][i]] = admittime[patients['SUBJECT_ID'][i]].year - patients['DOB'][i].year

### Combine

Based on subject_id, combine the variable we get.

In [157]:
# Based on the HADM_dict obtained from admissions, obtain icu_los from icustays dataframe
# One hospitalization may enter the ICU multiple times, and the length needs to be accumulated

icu_los = {}
for i in icustays.index:
    if icustays['HADM_ID'][i] == HADM_dict[icustays['SUBJECT_ID'][i]]:
        if icustays['SUBJECT_ID'][i] not in icu_los.keys():
            icu_los[icustays['SUBJECT_ID'][i]] = 0
        icu_los[icustays['SUBJECT_ID'][i]] += icustays['LOS'][i]

In [187]:
combine = convert_float[[(index in icu_los.keys()) for index in convert_float.index]]

In [188]:
combine['age'] = -1.0
combine['gender'] = -1
combine['icu_los'] = -1.0
combine['flag'] = -1

for i in combine.index:
    combine.age[i] = age_dict[i]
    combine.gender[i] = gender_dict[i]
    combine.icu_los[i] = time_dict[i]['icu_los']
    combine.flag[i] = flag_dict[i]

<ipython-input-188-f7a2a4fa7d25>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combine['age'] = -1.0
<ipython-input-188-f7a2a4fa7d25>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combine['gender'] = -1
<ipython-input-188-f7a2a4fa7d25>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [189]:
combine

ITEMID,50802,50804,50818,50820,50821,50868,50882,50893,50902,50912,...,51265,51274,51275,51277,51279,51301,age,gender,icu_los,flag
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,
3,-4,23,40,7.35,20,23,13,5.8,111,2.4,...,190,15.7,58.3,15.0,2.70,11.3,76.0,1,6.0646,0
9,1,29,46,7.39,75,13,26,9.2,100,1.4,...,249,13.5,21.9,13.8,4.50,13.7,41.0,1,5.3231,1
12,-21,10,36,7.02,276,28,11,8.3,111,1.3,...,87,16.8,60.9,14.2,4.40,8.4,72.0,1,7.6348,1
13,0,26,43,7.38,369,14,23,8.9,106,0.6,...,216,13.7,44.1,13.4,4.08,16.6,40.0,0,3.6660,0
17,-7,34,167,6.88,320,12,23,7.9,112,0.8,...,138,14.9,31.8,12.6,3.42,10.5,47.0,0,2.0710,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99946,4,30,42,7.45,94,15,26,9.1,105,0.8,...,231,30.9,39.0,13.8,5.63,15.4,50.0,1,0.7992,0
99982,8,33,40,7.51,100,13,30,8.8,98,1.5,...,113,14.7,31.4,15.2,3.18,3.8,65.0,1,7.9493,0
99985,0,26,39,7.41,39,12,24,7.2,104,0.9,...,250,12.5,47.7,14.2,3.18,12.6,54.0,1,11.2998,0


### Variables selection

There are too many variables, we want to choose 10 variables for the next part of data analysis. Therefore, we use a simple logistic regression to select variables.

In [190]:
# Map the data to the range of 0-1

combine_nor = combine.copy()
for column in combine_nor.columns:
    combine_nor[column] = combine_nor[column].astype('float') - min(combine_nor[column].astype('float'))
    combine_nor[column] = combine_nor[column] / max(combine_nor[column])

In [191]:
#Logistic regression

x_column = combine_nor.columns[:-1]
x = combine_nor[x_column]
y = combine_nor['flag']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)
clf = LogisticRegression(random_state=0, class_weight = 'balanced').fit(X_train, y_train)
clf.score(X_test,y_test)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6987647389107243

In [192]:
# The importance of variables

clf.coef_

array([[-2.1758029 , -2.73706951,  1.55998412,  0.54184627, -0.3229467 ,
         3.59556658, -0.37894219, -1.12112914, -4.30788353, -4.252456  ,
         1.81547318, -1.923065  , -0.50033385, -2.18868785,  3.00685438,
         2.89995361, -0.29202966,  0.66445185,  1.32347575,  1.32839141,
        -2.36666072,  1.98036039, -0.87288612,  1.21127145,  0.69200924,
         4.90876271,  0.67380674,  2.69264174,  1.2553999 , -0.0210873 ,
         1.73804051]])

In [193]:
x_column

Index([    50802,     50804,     50818,     50820,     50821,     50868,
           50882,     50893,     50902,     50912,     50931,     50960,
           50970,     50971,     50983,     51006,     51221,     51222,
           51237,     51248,     51249,     51250,     51265,     51274,
           51275,     51277,     51279,     51301,     'age',  'gender',
       'icu_los'],
      dtype='object', name='ITEMID')

In [194]:
# Drop the variables with low importance

drop_cols = [50820, 50821, 50882, 50970, 51221, 51222, 
             51265, 51275, 51279, 50818, 50893, 50804,
             51274, 51250, 51249, 51248, 51237, 50931, 
             50960,'gender','age']

combine_new = combine.drop(drop_cols, axis = 1)

In [195]:
# Map the data to the range of 0-1

combine_nor = combine_new.copy()
for column in combine_nor.columns:
    combine_nor[column] = combine_nor[column].astype('float') - min(combine_nor[column].astype('float'))
    combine_nor[column] = combine_nor[column] / max(combine_nor[column])

In [196]:
# Repeat the logistic regression

x_column = combine_nor.columns[:-1]
x = combine_nor[x_column]
y = combine_nor['flag']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)
clf = LogisticRegression(random_state=0, class_weight = 'balanced').fit(X_train, y_train)
clf.score(X_test,y_test)

0.7063447501403706

In [197]:
clf.coef_

array([[-4.95377852,  4.10662256, -3.96803642, -4.65643424, -2.22710977,
         3.08866454,  3.54293   ,  5.54482311,  2.77263174,  1.53964616]])

We can see that these ten variables have high importance. Therefore, we choose these ten variables for the next part of data analysis.

### Convert itemid and save dataframe

In [202]:
# Transfer item id to item name

itemid_dict = {}
for i in d_labitems.index:
    if d_labitems['ITEMID'][i] in combine_new.columns:
        itemid_dict[d_labitems['ITEMID'][i]] = d_labitems['LABEL'][i]
        
final_dataset = combine_new.rename(columns = itemid_dict)
final_dataset

ITEMID,Base Excess,Anion Gap,Chloride,Creatinine,Potassium,Sodium,Urea Nitrogen,RDW,White Blood Cells,icu_los,flag
SUBJECT_ID,,,,,,,,,,,
3,-4,23,111,2.4,4.0,143,41,15.0,11.3,6.0646,0
9,1,13,100,1.4,3.0,136,17,13.8,13.7,5.3231,1
12,-21,28,111,1.3,4.6,145,28,14.2,8.4,7.6348,1
13,0,14,106,0.6,3.5,139,13,13.4,16.6,3.6660,0
17,-7,12,112,0.8,3.8,140,11,12.6,10.5,2.0710,0
...,...,...,...,...,...,...,...,...,...,...,...
99946,4,15,105,0.8,3.2,143,20,13.8,15.4,0.7992,0
99982,8,13,98,1.5,3.8,137,29,15.2,3.8,7.9493,0
99985,0,12,104,0.9,3.9,136,13,14.2,12.6,11.2998,0


In [203]:
# Save to csv file

final_dataset.to_csv('death_risk_predict.csv')